In [2]:
########################################################################################################
# The RWKV Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=200)
import types, torch
from torch.nn import functional as F
from tokenizers import Tokenizer

In [3]:
tokenizer = Tokenizer.from_file("20B_tokenizer.json")

args = types.SimpleNamespace()
args.MODEL_NAME = '/data1/ckw/RWKV-4-Pile-430M-20220808-8066'
args.n_layer = 24
args.n_embd = 1024

context = "\nDataWhalechina is an organization founded at Shanghai Jiao Tong University that helps learners learn artificial intelligence."
NUM_TRIALS = 3
LENGTH_PER_TRIAL = 100
TEMPERATURE = 1.0
TOP_P = 0.85
########################################################################################################

### RWKV 的时间混合实现

在 RWKV 模型中，时间混合（Time Mixing）是一个关键步骤，用于处理输入序列随时间的变化。以下是 `time_mixing` 函数的详细公式说明和代码注释。

#### 公式说明

时间混合的核心思想是通过时间混合系数将当前输入与先前的状态混合，以生成新的键、值和门控信号。这一过程涉及如下步骤：

1. **混合输入**：
   - 对当前输入 \( x \) 和前一状态进行加权平均：
     $$ x_k = x \cdot \text{time\_mix\_k} + \text{state}[5i+1] \cdot (1 - \text{time\_mix\_k}) $$
     $$ x_v = x \cdot \text{time\_mix\_v} + \text{state}[5i+1] \cdot (1 - \text{time\_mix\_v}) $$
     $$ x_r = x \cdot \text{time\_mix\_r} + \text{state}[5i+1] \cdot (1 - \text{time\_mix\_r}) $$

2. **状态更新**：
   - 更新状态：
     $$ \text{state}[5i+1] = x $$

3. **计算门控信号**：
   - 使用 sigmoid 激活函数计算门控信号 \( r \)：
     $$ r = \sigma(\text{rw} @ x_r) $$

4. **计算键和值**：
   - 通过线性变换生成键 \( k \) 和值 \( v \)：
     $$ k = \text{kw} @ x_k $$
     $$ v = \text{vw} @ x_v $$

5. **加权和计算**：
   - 根据加权和公式计算加权和 \( wkv \)：
     $$ a = e1 \cdot aa + e2 \cdot v $$
     $$ b = e1 \cdot bb + e2 $$
     $$ \text{wkv} = a / b $$

代码如下：

```python
@torch.jit.script_method
def time_mixing(self, x, state, i:int, time_mix_k, time_mix_v, time_mix_r, time_first, time_decay, kw, vw, rw, ow):
    # 混合当前输入和先前的状态
    xk = x * time_mix_k + state[5*i+1] * (1 - time_mix_k)
    xv = x * time_mix_v + state[5*i+1] * (1 - time_mix_v)
    xr = x * time_mix_r + state[5*i+1] * (1 - time_mix_r)

    # 更新状态
    state[5*i+1] = x

    # 计算门控信号
    r = torch.sigmoid(rw @ xr)
    
    # 计算键和值
    k = kw @ xk
    v = vw @ xv

    # 从状态中读取先前的累积值
    aa = state[5*i+2]
    bb = state[5*i+3]
    pp = state[5*i+4]

    # 计算加权和的第一部分
    ww = time_first + k
    qq = torch.maximum(pp, ww)
    e1 = torch.exp(pp - qq)
    e2 = torch.exp(ww - qq)
    a = e1 * aa + e2 * v
    b = e1 * bb + e2
    wkv = a / b

    # 计算新的权重和状态
    ww = pp + time_decay
    qq = torch.maximum(ww, k)
    e1 = torch.exp(ww - qq)
    e2 = torch.exp(k - qq)
    state[5*i+2] = e1 * aa + e2 * v
    state[5*i+3] = e1 * bb + e2
    state[5*i+4] = qq

    # 计算最终的输出
    return ow @ (r * wkv)
```

### 详细解释

1. **混合输入**：
   - `xk`, `xv`, `xr` 是输入 `x` 与状态 `state` 的加权混合，分别用于计算键、值和门控信号。

2. **状态更新**：
   - 将当前输入 `x` 存储在状态数组中，供下一步计算使用。

3. **计算门控信号**：
   - 使用 `torch.sigmoid` 计算门控信号 `r`，它决定了多少信息将被传递。

4. **计算键和值**：
   - 使用矩阵乘法计算键 `k` 和值 `v`。

5. **加权和计算**：
   - 通过指数加权平均计算加权和 `wkv`，这涉及处理数值稳定性问题（通过 `torch.maximum` 和指数运算）。

6. **更新状态**：
   - 更新状态数组中的累积值，以便后续时间步使用。

7. **计算最终输出**：
   - 使用门控信号 `r` 和加权和 `wkv` 计算最终输出。

这样，通过逐步混合当前输入和先前的状态，RWKV 模型实现了时间序列数据的有效处理。

### RWKV 的通道混合（Channel Mixing）实现与代码注释

在 RWKV 模型中，通道混合（Channel Mixing）是另一个关键步骤，用于处理不同通道之间的信息交换。以下是 `channel_mixing` 函数的详细公式说明和代码注释。

#### 公式说明

通道混合的核心思想是通过通道混合系数将当前输入与先前的状态混合，以生成新的键和门控信号。这一过程涉及如下步骤：

1. **混合输入**：
   - 对当前输入 \( x \) 和前一状态进行加权平均：
     $$ x_k = x \cdot \text{time\_mix\_k} + \text{state}[5i+0] \cdot (1 - \text{time\_mix\_k}) $$
     $$ x_r = x \cdot \text{time\_mix\_r} + \text{state}[5i+0] \cdot (1 - \text{time\_mix\_r}) $$

2. **状态更新**：
   - 更新状态：
     $$ \text{state}[5i+0] = x $$

3. **计算门控信号**：
   - 使用 sigmoid 激活函数计算门控信号 \( r \)：
     $$ r = \sigma(\text{rw} @ x_r) $$

4. **计算键**：
   - 通过 ReLU 和平方变换生成键 \( k \)：
     $$ k = (\text{ReLU}(\text{kw} @ x_k))^2 $$

5. **计算输出**：
   - 使用门控信号和键计算最终的输出：
     $$ \text{output} = r \cdot (\text{vw} @ k) $$

代码如下：

```python
@torch.jit.script_method
def channel_mixing(self, x, state, i:int, time_mix_k, time_mix_r, kw, vw, rw):
    # 混合当前输入和先前的状态
    xk = x * time_mix_k + state[5*i+0] * (1 - time_mix_k)
    xr = x * time_mix_r + state[5*i+0] * (1 - time_mix_r)

    # 更新状态
    state[5*i+0] = x

    # 计算门控信号
    r = torch.sigmoid(rw @ xr)

    # 计算键，并通过ReLU和平方变换
    k = torch.square(torch.relu(kw @ xk))  # square relu, primer paper

    # 计算最终的输出
    return r * (vw @ k)
```


1. **混合输入**：
   - `xk`, `xr` 是输入 `x` 与状态 `state` 的加权混合，分别用于计算键和门控信号。

2. **状态更新**：
   - 将当前输入 `x` 存储在状态数组中，供下一步计算使用。

3. **计算门控信号**：
   - 使用 `torch.sigmoid` 计算门控信号 `r`，它决定了多少信息将被传递。

4. **计算键**：
   - 使用 `torch.relu` 计算键 `k`，然后进行平方变换以增加非线性特性。

5. **计算最终输出**：
   - 使用门控信号 `r` 和键 `k` 计算最终输出。

通过这些步骤，RWKV 模型实现了通道间的信息有效交换，增强了模型对输入数据的处理能力。

In [4]:
class RWKV_RNN(torch.jit.ScriptModule):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.eval() # set torch to inference mode
        
        w = torch.load(args.MODEL_NAME + '.pth', map_location='cpu')
        for k in w.keys():
            if      '.time_' in k: w[k] = w[k].squeeze()
            if '.time_decay' in k: w[k] = -torch.exp(w[k].float()) # the real time decay is like e^{-e^x}
            else: w[k] = w[k].float() # convert to f32 type
        
        self.w = types.SimpleNamespace() # set self.w from w
        self.w.blocks = {}
        for k in w.keys(): # example: "blocks.0.att.time_first" => self.w.blocks[0].att.time_first
            parts = k.split('.')
            last = parts.pop()
            here = self.w
            for p in parts:
                if p.isdigit():
                    p = int(p)
                    if p not in here: here[p] = types.SimpleNamespace()
                    here = here[p]
                else:
                    if not hasattr(here, p): setattr(here, p, types.SimpleNamespace())
                    here = getattr(here, p)
            setattr(here, last, w[k])

    def layer_norm(self, x, w):
        return F.layer_norm(x, (self.args.n_embd,), weight=w.weight, bias=w.bias)

    @torch.jit.script_method
    def channel_mixing(self, x, state, i:int, time_mix_k, time_mix_r, kw, vw, rw):
        xk = x * time_mix_k + state[5*i+0] * (1 - time_mix_k)
        xr = x * time_mix_r + state[5*i+0] * (1 - time_mix_r)
        state[5*i+0] = x
        r = torch.sigmoid(rw @ xr)
        k = torch.square(torch.relu(kw @ xk)) # square relu, primer paper
        return r * (vw @ k)

    @torch.jit.script_method
    def time_mixing(self, x, state, i:int, time_mix_k, time_mix_v, time_mix_r, time_first, time_decay, kw, vw, rw, ow):
        xk = x * time_mix_k + state[5*i+1] * (1 - time_mix_k)
        xv = x * time_mix_v + state[5*i+1] * (1 - time_mix_v)
        xr = x * time_mix_r + state[5*i+1] * (1 - time_mix_r)
        state[5*i+1] = x
        r = torch.sigmoid(rw @ xr)
        k = kw @ xk
        v = vw @ xv
        
        aa = state[5*i+2]
        bb = state[5*i+3]
        pp = state[5*i+4]
        ww = time_first + k
        qq = torch.maximum(pp, ww)
        e1 = torch.exp(pp - qq)
        e2 = torch.exp(ww - qq)
        a = e1 * aa + e2 * v
        b = e1 * bb + e2
        wkv = a / b
        ww = pp + time_decay
        qq = torch.maximum(ww, k)
        e1 = torch.exp(ww - qq)
        e2 = torch.exp(k - qq)
        state[5*i+2] = e1 * aa + e2 * v
        state[5*i+3] = e1 * bb + e2
        state[5*i+4] = qq
        return ow @ (r * wkv)

    def forward(self, token, state):
        with torch.no_grad():
            if state == None:
                state = torch.zeros(self.args.n_layer * 5, self.args.n_embd)
                for i in range(self.args.n_layer): state[5*i+4] = -1e30 # -infinity
            
            x = self.w.emb.weight[token]
            x = self.layer_norm(x, self.w.blocks[0].ln0)
            for i in range(self.args.n_layer):
                att = self.w.blocks[i].att
                x = x + self.time_mixing(self.layer_norm(x, self.w.blocks[i].ln1), state, i, 
                    att.time_mix_k, att.time_mix_v, att.time_mix_r, att.time_first, att.time_decay, 
                    att.key.weight, att.value.weight, att.receptance.weight, att.output.weight)
                ffn = self.w.blocks[i].ffn
                x = x + self.channel_mixing(self.layer_norm(x, self.w.blocks[i].ln2), state, i, 
                    ffn.time_mix_k, ffn.time_mix_r, 
                    ffn.key.weight, ffn.value.weight, ffn.receptance.weight)
            
            x = self.w.head.weight @ self.layer_norm(x, self.w.ln_out)
            return x.float(), state

##########################################################################################################

采样方法和v2、v3版本相比没有发生变化，代码做了一点优化调整而已。

In [6]:
def sample_logits(out, temperature=1.0, top_p=0.8):
    probs = F.softmax(out, dim=-1).numpy()
    sorted_probs = np.sort(probs)[::-1]
    cumulative_probs = np.cumsum(sorted_probs)
    cutoff = float(sorted_probs[np.argmax(cumulative_probs > top_p)])
    probs[probs < cutoff] = 0
    if temperature != 1.0:
        probs = probs.pow(1.0 / temperature)
    probs = probs / np.sum(probs)
    out = np.random.choice(a=len(probs), p=probs)
    return out

########################################################################################################

In [7]:
print(f'\nUsing CPU. Loading {args.MODEL_NAME} ...')
model = RWKV_RNN(args)


Using CPU. Loading /data1/ckw/RWKV-4-Pile-430M-20220808-8066 ...


In [8]:
print(f'\nPreprocessing context (slow version. see v2/rwkv/model.py for fast version)')
init_state = None
for token in tokenizer.encode(context).ids:
    init_out, init_state = model.forward(token, init_state)


Preprocessing context (slow version. see v2/rwkv/model.py for fast version)


In [9]:
for TRIAL in range(NUM_TRIALS):
    print(f'\n\n--[ Trial {TRIAL} ]-----------------', context, end="")
    all_tokens = []
    out_last = 0
    out, state = init_out.clone(), init_state.clone()
    for i in range(LENGTH_PER_TRIAL):
        token = sample_logits(out, TEMPERATURE, TOP_P)
        all_tokens += [token]
        tmp = tokenizer.decode(all_tokens[out_last:])
        if '\ufffd' not in tmp: # only print when we have a valid utf-8 string
            print(tmp, end="", flush=True)
            out_last = i + 1
        out, state = model.forward(token, state)       
print('\n')



--[ Trial 0 ]----------------- 
DataWhalechina is an organization founded at Shanghai Jiao Tong University that helps learners learn artificial intelligence. The machine learning solutions applied to the class are called Persona, which consist of several categories:

\begin{tabular}{|c|c|c|}
\hline
  Name   & Description  \\ \hline
\hline
  \end{tabular}

DataWhalechina organizes the data in two ways:

\begin{tabular}{|c|c|c|}
\hline
  \multicolumn{2}{|c}{

--[ Trial 1 ]----------------- 
DataWhalechina is an organization founded at Shanghai Jiao Tong University that helps learners learn artificial intelligence. The main goal is to allow learners to learn how to use artificial intelligence in an integrated fashion, by using both AI and deep learning techniques. Datawhalechina aims to teach AI algorithms from scratch and teach them from scratch to become competent with many algorithms that humans could not have.

Applications

Projects 
 DeeplearningAI : Encourage AI algorithms to bec

### 备注：RWKV 的Scaling Law（缩放定律）

RWKV 的缩放定律描述了模型性能随着各种因素变化的数学关系。这些因素包括模型大小（$N$）、数据集大小（$D$）或最优计算预算（$C_{\min}$）。缩放定律的重要性体现在以下两个方面：
1. **预测与规划**：它们允许我们在训练大型模型之前，通过插值和外推来预测和规划成本和性能。
2. **反馈与研究**：它们提供了关于模型失效情况下的重要反馈，指引未来研究方向。

#### 关键内容总结：
- **与之前的RNN研究对比**：之前的工作指出，LSTM不完全遵循与Transformer相同的对数线性缩放定律。然而，RWKV模型的训练结果表明，RWKV遵循与Transformer相同的一般缩放定律形式。
- **实验验证**：在[v4的论文](https://arxiv.org/abs/2305.13048)通过训练45个RWKV模型，验证了其损失与计算量之间的线性关系，线性拟合的 $r^2$ 值为0.994，即使外推一个数量级，拟合度仍然很好（$r^2$为0.875）。

这些结果显示了RWKV模型在缩放时的优越性和与Transformer相似的性能缩放行为。